<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

## Reddit API:
1. Go to https://www.reddit.com/prefs/apps
2. Click "create application"
3. Name: My Sentiment Example
4. Choose script
5. Leave *description* and *about url* blank
6. Add http://www.example.com/unused/redirect/uri to *redirect uri*

## Imports

In [1]:
import praw
import openai

In [2]:
reddit = praw.Reddit(client_id="TODO", # the App ID retreived from the reddit application
                     client_secret="TODO", # the secret retreived from the reddit application
                     user_agent="sentiment analysis test (by u/YourUserName)"  # add your reddit name
                    )

We can now already use our reddit instance to obtain public information from reddit!

In [3]:
for submission in reddit.subreddit("test").hot(limit=10):
    print(submission.title)


test
test
Test
This is a test 1̶̳͖̍̂1̵̢͍͆1̴̫̚1̵͕̕1̶͖͎͋̊1̷̢͇͑̄1̵̢̱̈́1̷̖̐͝ͅ1̶͎̈́͊1̶̩̆̓1̶̺̀̉1̶̺̟̀͝1̸͈̐1̸͖̆1̷̭̋̔1̸̺̾1̸̢̃͠1̸̗̿̾1̶̨̖̌1̸̜̓͜
This is a test
This is a test
What are you doing? | Nafantik 🐾
-
Test
test from client


Of course we want to access hot subreddits like r/wallstreetbets or r/stocks

In [4]:
subreddit_stocks = reddit.subreddit("stocks")

In [5]:
print(subreddit_stocks.display_name)
print(subreddit_stocks.title)
print(subreddit_stocks.accounts_active)

stocks
Stocks - Investing and trading for all
5977


Let's say we want to access the first 5 hot posts and three most upvoted comments:

In [6]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title)
    submission = reddit.submission(post.id)
    c = 0
    for comment in submission.comments:
        comment = comment.body
        if not comment == "[deleted]":
            print(comment)
            c+=1
        if c == 2:
            break

Rate My Portfolio - r/Stocks Quarterly Thread December 2022
Brokerage account:

69% Vanguard FTSE All-World ETF ( +1% on invested capital).

31% GOOGL (+7% on invested capital).

A bit nervous about GOOGL. Might sell some shares and buy more ETF.
Overall: 

30% VOO 30% BRK.B 40% Individual Stocks 

Individual Picks: 
AAPL
MSFT
SBUX
NFLX
SHOP
NKE
FB
GOOG
NVDA
/r/Stocks Weekend Discussion Saturday - Feb 04, 2023
First
Musk found not liable in securities class action suit
This investor has suffered so much harm. Stock up only 800% since that fraud tweet. 

So much irreparable damage.
No dog in this fight outside of owning tesla shares indirectly from index funds, but what would stop a company, say Jones Soda (JSDA) from saying Saudi Arabia wants to take them private at a 30% premium. 

Does this jury verdict reset what the norm is for corporate finance- from an adjudicating stand point? Being able to tweet/email/DM that you're being purchased to increase market cap with no repercussions?


CPI is reweighting every year instead of two years. Whether CPI will be higher or lower depends on whether consumer spending has shifted more towards goods and services with higher inflation or to goods and services with lower inflation.

Math example:

The baseline for prices is 100 and that was set as the average price for that good from 1982-84.

Let's say the economy only has two goods FOOD and CLOTHING. The baseline from 82-84 would be 100 for FOOD and 100 for CLOTHING. Let's assume people spend 50% of their income on FOOD and 50% on CLOTHING. Multiply 0.50 x 100 and 0.50 x 100 and add them together to get an overall price level of 100.

At some point in the future FOOD has a price level of 120 and CLOTHING has a price level of 150. CLOTHING is really expensive now. If you used the old weighting of 50% for each then you'd multiply 0.50 x 120 and 0.50 x 150 and add them together to get 135 as the new CPI. That would mean inflation of 35%.

However, let's say that people only spent 

We can now use GPT to extract stock tickers from all of these posts and tell us whether the sentiment for the mentioned assets is positive, neutral or negative:

## OpenAI API

Remember to use the notebook as shown, you must set your OpenAI API Key as an environment variable. Obviously, there are many ways you could provide your API Key to the Python code, input() or even hard-coded, but those are typically not recommended for safety reasons. Having it as an environment variable let's the key live on the computer, but not actually be present in the code.

### Set-up Open AI API Key

We'll only need to do this once per computer

In [ ]:
# Uncomment below and swap in your key to place your environment key using Python
# Then you can delete the key string and the code cell below will still work!
# os.environ["OPENAI_API_KEY"] = "Your key goes here!

In [23]:
openai.api_key = os.getenv("OPENAI_API_KEY")

## Prompt Generation
First, we need to create a dictionary containing the title and top three comments

In [ ]:
getattr(subreddit_stocks, "hot")

In [10]:
def get_titles_and_comments(subreddit="stocks", sub_instance="hot", limit=10, num_comments=2, skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    titles_and_comments = {}
    for c, post in enumerate(getattr(subreddit, sub_instance)(limit=limit)):
        
        if c < skip_first:
            continue
        
        c+=(1-skip_first)
        
        titles_and_comments[c] = ""

        submission = reddit.submission(post.id)
        title = post.title
        
        titles_and_comments[c] += "Title: " + title  + "\n\n"
        titles_and_comments[c] += "Comments: \n\n"
        
        comment_counter = 0
        for comment in submission.comments:
            comment = comment.body
            if not comment == "[deleted]":
                titles_and_comments[c] += comment + "\n"
                comment_counter+=1
            if comment_counter == num_comments:
                break

    return titles_and_comments

In [14]:
titles_and_comments = get_titles_and_comments(subreddit="stocks", limit=12)

In [15]:
print(titles_and_comments[2])

Title: December jobs blowout: +517k added (vs +187k expected); unemployment rate hits 3.4%

Comments: 

Insane... Even in a high rates environment

I wonder how the fed responds in medium term
Holy balls lol



In [16]:
def create_prompt(title_and_comments):

    task = "Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':\n\n"
    return task + title_and_comments


In [20]:
prompt = create_prompt(titles_and_comments[1])

In [21]:
print(prompt)

Return the stock ticker or company in the following heading and comments and classify the sentiment. If no ticker or company is mentioned write 'No company mentioned':

Title: Musk found not liable in securities class action suit

Comments: 

This investor has suffered so much harm. Stock up only 800% since that fraud tweet. 

So much irreparable damage.
No dog in this fight outside of owning tesla shares indirectly from index funds, but what would stop a company, say Jones Soda (JSDA) from saying Saudi Arabia wants to take them private at a 30% premium. 

Does this jury verdict reset what the norm is for corporate finance- from an adjudicating stand point? Being able to tweet/email/DM that you're being purchased to increase market cap with no repercussions?



In [26]:
for key, title_and_comments in titles_and_comments.items():
    
    prompt = create_prompt(title_and_comments)
    
    response = openai.Completion.create(engine="text-davinci-003",
                                                prompt=prompt,
                                                max_tokens=256,
                                                temperature=0,
                                                top_p=1.0,
                                                frequency_penalty=0.0,
                                                presence_penalty=0.0)
    print(title_and_comments)
    print("Sentiment: " + response["choices"][0]["text"])
    print("-"*30)

Title: Musk found not liable in securities class action suit

Comments: 

This investor has suffered so much harm. Stock up only 800% since that fraud tweet. 

So much irreparable damage.
No dog in this fight outside of owning tesla shares indirectly from index funds, but what would stop a company, say Jones Soda (JSDA) from saying Saudi Arabia wants to take them private at a 30% premium. 

Does this jury verdict reset what the norm is for corporate finance- from an adjudicating stand point? Being able to tweet/email/DM that you're being purchased to increase market cap with no repercussions?

Sentiment: 
No company mentioned
------------------------------
Title: December jobs blowout: +517k added (vs +187k expected); unemployment rate hits 3.4%

Comments: 

Insane... Even in a high rates environment

I wonder how the fed responds in medium term
Holy balls lol

Sentiment: 
No company mentioned
------------------------------
Title: Amazon, Meta, Alphabet, and Microsoft will incur $10B+ 